In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /Users/jds/opt/anaconda3/lib/python3.8/site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [8]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

# Read the CSV and Perform Basic Data Cleaning

In [9]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [30]:
df = df[df.columns.drop(list(df.filter(regex='err')))]
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,...,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,162.513840,0.586,4.50700,874.8,...,443,9.11,25.8,2,5455,4.467,0.927,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,175.850252,0.969,1.78220,10829.0,...,638,39.30,76.3,1,5853,4.544,0.868,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,170.307565,1.276,2.40641,8079.2,...,1395,891.96,505.6,1,5805,4.564,0.791,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,171.595550,0.701,1.65450,603.3,...,1406,926.16,40.9,1,6031,4.438,1.046,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,172.979370,0.762,3.14020,686.0,...,1160,427.65,40.2,2,6046,4.486,0.972,296.28613,48.224670,15.714


# Select your features (columns)

In [83]:
y = df['koi_disposition']
#target_names = ["CONFIRMED", "FALSE POSITIVE", "CANDIDATE"]
X = df.drop("koi_disposition", axis=1)


# Create a Train Test Split

Use `koi_disposition` for the y values

In [84]:
y = df['koi_disposition']
print(X.shape, y.shape)

(6991, 20) (6991,)


In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [88]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [89]:
labelencoder.fit(y_train)

LabelEncoder()

In [90]:
y_train_encoded = labelencoder.transform(y_train)

In [91]:
y_test_encoded= labelencoder.transform(y_test)

In [92]:
y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [93]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler 
X_scaler = MinMaxScaler().fit(X_train)

In [94]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [95]:
X_train

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
3563,0,0,0,0,10.548413,139.064020,1.0170,1.8720,102.9,3.89,899,154.84,11.7,2,6047,4.387,1.092,298.09543,44.737061,13.204
4099,0,0,0,0,24.754385,140.207320,0.7090,3.3900,593.3,2.10,491,13.70,18.0,1,4852,4.519,0.804,295.73535,42.576248,15.514
5460,0,0,0,0,1.057336,131.792007,0.2620,1.5795,47337.0,14.59,1276,623.51,476.0,1,4664,4.594,0.683,292.18417,49.310040,15.414
1091,0,0,0,0,201.118319,187.569860,0.0010,10.3280,584.8,2.28,300,1.92,34.7,1,5646,4.447,0.954,283.11377,48.131390,13.328
5999,0,0,0,0,91.649983,175.715600,0.2136,10.2940,193.6,2.27,568,24.57,8.7,2,6705,4.145,1.608,294.93198,39.812420,12.964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,0,0,0,0,52.069002,178.155250,0.2750,3.0089,558.3,2.92,524,17.76,25.6,1,5565,4.251,1.246,289.67935,40.700459,14.279
5192,1,0,0,0,2.387937,133.229020,1.1930,17.3020,114.3,25.56,1674,1860.02,41.7,1,6769,4.379,1.161,296.40967,43.876019,14.394
3980,0,0,1,0,1.070273,131.835891,1.2690,4.2844,9886.5,28.73,1224,528.52,284.0,1,4628,4.651,0.653,299.17062,40.871059,15.926
235,0,0,0,0,18.746490,147.130740,0.5080,3.8950,796.8,3.05,686,52.38,26.7,2,5561,4.362,1.060,288.16470,45.816509,15.843


In [96]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation="relu", input_dim=20))
model.add(Dense(units=100, activation="relu"))
model.add(Dense(units=3, activation="softmax"))

In [97]:
X_train_scaled.shape

(5243, 20)

In [98]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 100)               2100      
_________________________________________________________________
dense_10 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 303       
Total params: 12,503
Trainable params: 12,503
Non-trainable params: 0
_________________________________________________________________


In [99]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [100]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
164/164 - 3s - loss: 0.5332 - accuracy: 0.7191
Epoch 2/60
164/164 - 0s - loss: 0.3942 - accuracy: 0.7814
Epoch 3/60
164/164 - 0s - loss: 0.3822 - accuracy: 0.7946
Epoch 4/60
164/164 - 0s - loss: 0.3787 - accuracy: 0.7936
Epoch 5/60
164/164 - 0s - loss: 0.3738 - accuracy: 0.7974
Epoch 6/60
164/164 - 0s - loss: 0.3724 - accuracy: 0.7999
Epoch 7/60
164/164 - 0s - loss: 0.3665 - accuracy: 0.8035
Epoch 8/60
164/164 - 0s - loss: 0.3659 - accuracy: 0.8043
Epoch 9/60
164/164 - 0s - loss: 0.3633 - accuracy: 0.8091
Epoch 10/60
164/164 - 0s - loss: 0.3615 - accuracy: 0.8097
Epoch 11/60
164/164 - 0s - loss: 0.3593 - accuracy: 0.8125
Epoch 12/60
164/164 - 0s - loss: 0.3572 - accuracy: 0.8159
Epoch 13/60
164/164 - 0s - loss: 0.3566 - accuracy: 0.8119
Epoch 14/60
164/164 - 0s - loss: 0.3559 - accuracy: 0.8131
Epoch 15/60
164/164 - 0s - loss: 0.3555 - accuracy: 0.8116
Epoch 16/60
164/164 - 0s - loss: 0.3489 - accuracy: 0.8173
Epoch 17/60
164/164 - 0s - loss: 0.3495 - accuracy: 0.8135
Epoch 

In [101]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.3477 - accuracy: 0.8507
Normal Neural Network - Loss: 0.3477300703525543, Accuracy: 0.8506864905357361


# Train the Model



In [102]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.3477 - accuracy: 0.8507
Normal Neural Network - Loss: 0.3477300703525543, Accuracy: 0.8506864905357361


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

In [28]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 1, 'gamma': 0.0001}
0.7354568873299907


# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)